<a href="https://colab.research.google.com/github/hossein20s/PPG2bloodPressure/blob/master/data_io_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
Version = '1.4'
print('data.io.lib Version: ', Version)

In [0]:
import pandas as pd

def normalize(df):
  #stats = dataframe.describe()
  return (df - df.mean()) / df.std()


def get_normalised_column(column):
                x = column[0]
                if(x == 0):
                  x = window[1]
                  print(col_i, x)
                normalised_col = [((float(p) / float(x)) - 1) for p in column]
                return normalised_col

def get_normalized_column(column):
  df = pd.DataFrame(column)
  normalized_col = normalize(df).values
  normalized_col = normalized_col.reshape(normalized_col.shape[0])
  return normalized_col



In [0]:
import math
import numpy as np
import pandas as pd
import h5py


class DataLoader():
    """A class for loading and transforming data for the lstm model"""

    def __init__(self, filename, split, cols, index=None):
      if(filename.endswith('.csv')):
        dataframe = pd.read_csv(filename)
      elif(filename.endswith('.mat')):
        with h5py.File(filename, 'r') as f:
          print(list(f.keys()))
          tmp = f['Part_1']
          objectRef = tmp[0][0]
          ds = f[objectRef]
          dataframe = pd.DataFrame(index=index, columns=cols)
          print(len(cols))
          for i in range(len(cols)):
            print(cols[i])
            dataframe[cols[i]] = ds[:,i]
          print(dataframe.shape)
          dataframe = dataframe[:3000]
          print(dataframe.shape)
          print(dataframe.head(3))

      i_split = int(len(dataframe) * split)
      self.data_train = dataframe.get(cols).values[:i_split]
      self.data_test  = dataframe.get(cols).values[i_split:]
      self.len_train  = len(self.data_train)
      self.len_test   = len(self.data_test)
      self.len_train_windows = None

    def get_test_data(self, seq_len, normalise):
        '''
        Create x, y test data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise reduce size of the training split.
        '''
        data_windows = []
        for i in range(self.len_test - seq_len):
            data_windows.append(self.data_test[i:i+seq_len])

        data_windows = np.array(data_windows).astype(float)
        data_windows = self.normalise_windows(data_windows, single_window=False) if normalise else data_windows

        x = data_windows[:, :-1]
        y = data_windows[:, -1, [0]]
        return x,y

    def get_train_data(self, seq_len, normalise):
        '''
        Create x, y train data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise use generate_training_window() method.
        '''
        data_x = []
        data_y = []
        for i in range(self.len_train - seq_len):
            x, y = self._next_window(i, seq_len, normalise)
            data_x.append(x)
            data_y.append(y)
        return np.array(data_x), np.array(data_y)

    def generate_train_batch(self, seq_len, batch_size, normalise):
        '''Yield a generator of training data from filename on given list of cols split for train/test'''
        i = 0
        while i < (self.len_train - seq_len):
            x_batch = []
            y_batch = []
            for b in range(batch_size):
                if i >= (self.len_train - seq_len):
                    # stop-condition for a smaller final batch if data doesn't divide evenly
                    yield np.array(x_batch), np.array(y_batch)
                    i = 0
                x, y = self._next_window(i, seq_len, normalise)
                x_batch.append(x)
                y_batch.append(y)
                i += 1
            yield np.array(x_batch), np.array(y_batch)

    def _next_window(self, i, seq_len, normalise):
        '''Generates the next data window from the given index location i'''
        window = self.data_train[i:i+seq_len]
        window = self.normalise_windows(window, single_window=True)[0] if normalise else window
        x = window[:-1]
        y = window[-1, [0]]
        return x, y

    def normalise_windows(self, window_data, single_window=False):
        '''Normalise window with a base value of zero'''
        normalised_data = []
        window_data = [window_data] if single_window else window_data
        for window in window_data:
            normalised_window = []
            for col_i in range(window.shape[1]):
                normalised_col = get_normalized_column(window[:,col_i])
                normalised_window.append(normalised_col)
            normalised_window = np.array(normalised_window).T # reshape and transpose array back into original multidimensional format
            normalised_data.append(normalised_window)
        return np.array(normalised_data)
